# Implementation of a decision tree without sklearn library

## Importing libraries 

In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn import datasets
import math

## Loading datasets 

In [2]:
iris = datasets.load_iris()
print(iris)

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

In [3]:
X = iris.data
Y = iris.target

In [4]:
df = pd.DataFrame(X)
df

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


## Data cleaning and exploratory data analysis

In [5]:
df.columns = ['sl', 'sw', 'pl', 'pw']
df

,sl,sw,pl,pw
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [6]:
df.describe()

,sl,sw,pl,pw
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [7]:
df.isna().sum()

sl    0
sw    0
pl    0
pw    0
dtype: int64

## Discretisation of data  

In [8]:
def toLabel(vals, *args):
    if vals < args[0]:
        return 'a'
    elif vals < args[1]:
        return 'b'
    elif vals < args[2]:
        return 'c'
    else:
        return 'd'
def discretization(df, old_feature):
    min_value = df[old_feature].min()
    mean_value = df[old_feature].mean()
    max_value = df[old_feature].max()
    first = (min_value+mean_value)/2
    second = (max_value+mean_value)/2
    return df[old_feature].apply(toLabel, args=(first, mean_value, second))

In [9]:
df['sepal_length'] = discretization(df, 'sl')
df['sepal_width'] = discretization(df, 'sw')
df['petal_length'] = discretization(df, 'pl')
df['petal_width'] = discretization(df, 'pw')
df

,sl,sw,pl,pw,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [10]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis=1, inplace=True)
df

,sepal_length,sepal_width,petal_length,petal_width
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a
...,...,...,...,...
145,c,b,c,d
146,c,a,c,d
147,c,b,c,d
148,c,c,d,d


## Decision tree classifier 

In [11]:
class decisionTree:
    def __init__(self, data, output):
        #feature on which the split is made
        self.data = data
        #dictionary to store children of the current node in the form of key value pair 
        self.children = {}
        #self.output is used to store the majority feature at the particular node of decision tree
        self.output = output
        
    def add_child(self, feature_value, node):
        self.children[feature_value] = node

In [12]:
class build_tree:
    def __init__(self):
        self.root = None
        
    #calculate the frequency of each output class
    def frequency_count(self, y):
        d = {}
        for i in y:
            if i in d:
                d[i] += 1
            else:
                d[i] = 1
        return d
    
    #entropy of each node 
    def entropy_feature(self, y):
        freq = self.frequency_count(y)
        entropy = 0
        total = len(y)
        for i in freq:
            prob = freq[i]/total
            entropy += (-prob)*math.log2(prob)

        return entropy
    
    #function to calculate the gain ratio of each node 
    def gain_ratio(self, x, y, feature):
        entropy_before_split = self.entropy_feature(y)
        entropy_after_split = 0
        split_info = 0
        values_in_feature = set(x[:, feature])
        df3 = pd.DataFrame(x)
        df3[df3.shape[1]] = y
        starting_size = df3.shape[0]

        #iteration over each feature of the particular split 
        for i in values_in_feature:
            df3_new = df3[df3[feature]==i]

            after_split_size = df3_new.shape[0]
            entropy_after_split += (after_split_size/starting_size)*(self.entropy_feature(df3_new[df3_new.shape[1]-1]))

            split_info += (-after_split_size/starting_size)*math.log2(after_split_size/starting_size)
        if split_info == 0 :
            return -math.inf
        info_gain = entropy_before_split - entropy_after_split
        gain = info_gain/split_info

        return gain
            
        
    def decision_tree(self, x, y, features, classes, level):
        all_feature = [i for i in df.columns]
        
        #checking the stopping criteria for the code 
        #all features are exhausted
        if len(features)==0:
            print('Level', level)
            freq = self.frequency_count(y)
            max_count = -1
            output = None 
            for i in classes:
                if i not in freq:
                    print('Count', i, '=', 0)
                else:
                    print('Count', i, '=', freq[i])
                    if freq[i]>max_count:
                        max_count = freq[i]
                        output = i 
            print('Current node entropy is', self.entropy_feature(y))
            print('Reached leaf node')
            print()
            return decisionTree(None, output)
        
        #if only one value is present 
        if len(set(y))==1:
            print('Level', level)

            output = None 
            for i in classes:
                if i not in y:
                    print('Count', i, '=', 0)
                else:
                    print('Count', i, '=', len(Y))
                    output = i
            print('Current node entropy is ', 0.0)
            print('Reached leaf node')
            print()
            return decisionTree(None, output)
        
        #checking the gain ratio of each feature and selecting the feature with maximum gain
        max_gain = -1
        selected_feature = None
        for feature in features:
            gain = self.gain_ratio(x, y, feature)
            if max_gain < gain:
                max_gain = gain 
                selected_feature = feature
        
        #printing of tree node
        print('Level', level)
        freq = self.frequency_count(y)
        output = None 
        max_count = -1
        for i in classes:
            if i not in freq:
                print('Count', i, '=', 0)
            else:
                print('Count', i, '=', freq[i])
                if max_count < freq[i]:
                    max_count = freq[i]
                    output = i
        print('Current node entropy is', self.entropy_feature(y))
        
        print('Tree is getting split for the feature', all_feature[selected_feature], 'gain ratio of', max_gain)
        print()
        
        #after the split checking for the next feature on which the spliting of next level will be done 
        unique_values = set(x[:, selected_feature])
        df2 = pd.DataFrame(x)
        df2[df.shape[1]] = y
        current_node = decisionTree(selected_feature, output)
        index  = features.index(selected_feature)
        features.remove(selected_feature)
        for i in unique_values:
            df_new = df2[df2[selected_feature]==i]
            node = self.decision_tree(df_new.iloc[:, 0:df_new.shape[1]-1].values, df_new.iloc[:, df_new.shape[1]-1].values, features, classes,level+1)
            current_node.add_child(i, node)
        return current_node
    
    #fitting of testing data and formation of decision tree
    def fit(self, x, y):
        features = [i for i in range(len(x[0]))]
        classes = set(y)
        level = 0
        self.root = self.decision_tree(x, y, features, classes, level)
    
    def prediction_for_row(self, X, node):
        if len(node.children)==0:
            return node.output
        
        val = X[node.data]
        if val not in node.children:
            return node.output
        
        return self.prediction_for_row(X, node.children[val])
            
        
    def prediction(self, X):
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.prediction_for_row(X[i], self.root)
        return Y
    
    def score(self, x_test, y_test):
        y_pred = self.prediction(x_test)
        correct = 0
        for i in range(len(y_test)):
            if y_pred[i] == y_test[i]:
                correct += 1
        return correct/len(y_test)

In [13]:
x = df.values
y = Y
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42)
clf = build_tree()
clf.fit(x_train, y_train)

Level 0
Count 0 = 35
Count 1 = 39
Count 2 = 38
Current node entropy is 1.5834545859901241
Tree is getting split for the feature petal_length gain ratio of 0.68795925549075

Level 1
Count 0 = 0
Count 1 = 0
Count 2 = 150
Current node entropy is  0.0
Reached leaf node

Level 1
Count 0 = 150
Count 1 = 0
Count 2 = 0
Current node entropy is  0.0
Reached leaf node

Level 1
Count 0 = 0
Count 1 = 150
Count 2 = 0
Current node entropy is  0.0
Reached leaf node

Level 1
Count 0 = 0
Count 1 = 33
Count 2 = 15
Current node entropy is 0.8960382325345575
Tree is getting split for the feature petal_width gain ratio of 0.3675534153001154

Level 2
Count 0 = 0
Count 1 = 0
Count 2 = 150
Current node entropy is  0.0
Reached leaf node

Level 2
Count 0 = 0
Count 1 = 150
Count 2 = 0
Current node entropy is  0.0
Reached leaf node

Level 2
Count 0 = 0
Count 1 = 30
Count 2 = 7
Current node entropy is 0.6997722217733069
Tree is getting split for the feature sepal_length gain ratio of 0.1469861041541423

Level 3
Cou

## Checking for accuracy 

In [14]:
clf.score(x_test, y_test)

0.9736842105263158